In [5]:
import pandas as pd
from sqlalchemy import create_engine
from urllib.parse import quote

sqlEngine = create_engine(
    "mysql+pymysql://root:%s@172.16.10.112:3306/hkd" % quote("Ptdl@123")
)
sqlEngine = create_engine(
    "mysql+pymysql://root:%s@127.0.0.1:3306/bid" % quote("")
)

provinces_info = pd.read_csv("gdt_province.csv")

In [7]:
# Start import city
city = pd.DataFrame()

city['name'] = provinces_info['tinh'].drop_duplicates()
city['status'] = 1

with sqlEngine.begin() as connection:
        city.to_sql(
            name="provinces", con=connection, if_exists="append", index=False
        )

In [19]:
def get_id_city(name, list_city):
	return list_city[name]

In [24]:
query = "SELECT id, `name` FROM provinces WHERE head_id IS NULL"
sql_city_info = pd.read_sql(query, con=sqlEngine)
sql_city_info = dict(zip(sql_city_info.name, sql_city_info.id))
provinces_info['city_id'] = provinces_info['tinh'].apply(lambda x: sql_city_info[x])

district = pd.DataFrame()
district["name"] = provinces_info["huyen"].drop_duplicates()
district['status'] = 1
district['head_id'] = provinces_info['city_id'] 

with sqlEngine.begin() as connection:
        district.to_sql(
            name="provinces", con=connection, if_exists="append", index=False
        )

In [26]:
query = "SELECT id, `name` FROM provinces WHERE head_id IS NOT NULL"
sql_district_info = pd.read_sql(query, con=sqlEngine)
sql_district_info = dict(zip(sql_district_info.name, sql_district_info.id))
provinces_info['district_id'] = provinces_info['huyen'].apply(lambda x: sql_district_info[x])
provinces_info.head()

commune = pd.DataFrame()
commune["name"] = provinces_info["xa"].drop_duplicates()
commune['status'] = 1
commune['head_id'] = provinces_info['district_id'] 

with sqlEngine.begin() as connection:
        commune.to_sql(
            name="provinces", con=connection, if_exists="append", index=False
        )